# 🚀 Proyecto ELT con SpaceX API  
## Notebook 02 – Transformación de datos

En este notebook se aplican tareas de **procesamiento y transformación de datos con Pandas**, según la consigna:  

- Eliminación de duplicados.  
- Manejo de valores nulos.  
- Renombrado de columnas.  
- Conversión de tipos.  
- Creación de nuevas columnas.  
- Unión (JOIN) entre DataFrames.  


In [1]:
import sys

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

# --- IMPORTS ---
from src.transform import load_from_parquet, drop_duplicates, handle_nulls, rename_columns, create_new_column, normalize_columns,  expand_payload_weights
from src.load import save_to_parquet 
from src.utils import setup_logger

logger = setup_logger()

# --- LOGGER ---
logger = setup_logger()
logger.info("Inicio de transformaciones de datos")


ImportError: cannot import name 'create_new_column' from 'src.transform' (c:\Users\MONSO\OneDrive\Escritorio\Final-DataEngineering\notebooks\..\src\transform.py)

## 📥 Lectura de datos

Se extraen los datasets desde Delta Lake.  
Trabajaremos principalmente con:  
- **Próximos lanzamientos** (`upcoming_launches`)  
- **Cohetes** (`rockets`)  


In [ ]:
# --- Extracción de datos para transformar --- Cargar datos desde Bronze
logger.info("Cargando Rockets desde Bronze...")
rockets_df = load_from_parquet("rockets", layer="bronze")
display(rockets_df.head())

logger.info("✅ Datos cargados desde Parquet. Inicio de transformaciones de datos.")

## 🛠 Transformaciones aplicadas
1. **Eliminar duplicados**.  
2. **Rellenar valores nulos** en la columna `details`.  
3. **Renombrar columnas** para mayor claridad (`date_utc → fecha_lanzamiento`).  
4. **Conversión de tipos** de fecha.  
5. **Nueva columna**: indicador booleano si el lanzamiento es en 2025.  
6. **Join** entre lanzamientos y cohetes.  


In [ ]:
# Normalizo para evitar problemas de incompatibilidad con arrays
rockets_df_normalized = normalize_columns(rockets_df)

# Aplicar transformaciones
logger.info("Aplicando transformaciones...")
rockets_df_normalized = drop_duplicates(rockets_df_normalized)
rockets_df_normalized = handle_nulls(rockets_df_normalized, ["description"], fill_value="Sin descripción")
rockets_df_normalized = rename_columns(rockets_df_normalized, {"name": "rocket_name"})
rockets_df_normalized = expand_payload_weights(rockets_df_normalized)
display(rockets_df_normalized.head())

logger.info("✅ Transformaciones aplicadas correctamente.")

## 💾 Guardado de datos normalizados y transformados en Silver

In [ ]:
# Guardar en Silver
logger.info("Guardando transformaciones en Silver...")
save_to_parquet(rockets_df, "rockets", layer="silver", incremental=False, mode="overwrite")

print(f"✅ Transformaciones guardadas en Silver: {len(rockets_df)} filas")

## 👀 Vista de resultados

Mostramos algunas columnas seleccionadas del DataFrame final:  
